# Question word prediction

> Group 12: Tristan Perrot & Romain Darous

Task is to train and evaluate a **char per char Transformer model** model using any available QA-corpus, for instance, the [SQuAD corpus](https://rajpurkar.github.io/SQuAD-explorer/).


METTRE EN CONTEXTE LE DEBUT DE LA QUESTION EN CORRIGEANT LE CODE DEJA FAIT
ADAPTER POUR AUGMENTER LE NOMBRE DE TRANSFORMERS EVENTUELLEMENT ET FAIRE PLUSIEURS COUCHES

# 0. Importing modules

In [1]:
import json
import math
import os

# Importing
import matplotlib.pyplot as plt
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange
import random
from datetime import datetime
import numpy as np

# Importing models
import char_dataset
import cpc_model


/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# 1. Data pre-processing

## 1.1. Loading the dataset
**Note :** we only want to be able te recover the beginning of a question. For that, it doesn"t matter whether the question is impossible to answer or not.

In [2]:
data_dir = 'data'
if data_dir not in os.listdir():
    os.mkdir(data_dir)

if "squad_train.json" not in os.listdir(data_dir):
    # Download data at https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
    res = requests.get(
        "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")
    data = json.loads(res.text)

    # Save data to file
    with open(data_dir + "/squad_train.json", "w") as f:
        json.dump(data, f)

with open(data_dir + "/squad_train.json", "r") as f:
    data = json.load(f)

# Extract answer text and question text
answers = []
questions = []
for article in data["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            
            """if qa["is_impossible"] :
                continue
            answers.append(qa["answers"][0]["text"])"""
                
            if qa["is_impossible"] and len(qa["question"]) > 4:
                continue
            else :
                answers.append("")
            questions.append(qa["question"])
            

print("Number of questions:", len(questions))

# Print some examples
for i in range(5):
    print()
    print("Question:", questions[i])
    print("Answer:", answers[i])

Number of questions: 86821

Question: When did Beyonce start becoming popular?
Answer: 

Question: What areas did Beyonce compete in when she was growing up?
Answer: 

Question: When did Beyonce leave Destiny's Child and become a solo singer?
Answer: 

Question: In what city and state did Beyonce  grow up? 
Answer: 

Question: In which decade did Beyonce become famous?
Answer: 


In [3]:
print(questions[i] + ' ' + answers[i])

In which decade did Beyonce become famous? 


## 1.2. Making a suitable dataset
``<BOS>`` token. Indicates that the sentence is starting.

We will make the prediction of the sentence in reverse mode, as we want to predict the beginning of a question. We will use unidirectionnal attention as well.

In [4]:
# Concatenating questions and answers
dataset = [(questions[i].lower() + ' ' + answers[i].lower())[::-1] for i in range(len(questions))]

# Shuffle dataset
random.shuffle(dataset)

# Splitting into train, validation, and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size = int(0.1*len(dataset)), train_size=int(0.9*len(dataset)))
train_dataset, val_dataset = train_test_split(train_dataset, train_size=int(0.85*len(train_dataset)), test_size = int(0.15*len(train_dataset)))

In [5]:
print(f"Size of the dataset : {len(dataset)}")
print(f"Size of the train, val and test sets : {len(train_dataset), len(val_dataset), len(test_dataset)}")
print(f"Example of original datapoint : {questions[0] + ' ' + answers[0]}")
print(f"Example of formatted datapoint : {dataset[0]}")

Size of the dataset : 86821
Size of the train, val and test sets : (66417, 11720, 8682)
Example of original datapoint : When did Beyonce start becoming popular? 
Example of formatted datapoint :  ?0102 fo yam ni desiver niaga saw tahw


## 1.3. Building a character dataset
The dataset will be built in the ``train_charlm`` function as it depends on the desired configuration. The function that allow to build the dataset and their context are provided in the file ``char_dataset.py``.

# 2. The model

In [6]:
# Updating models# Delete the modules from the namespace
del char_dataset
del cpc_model

# Unload the modules from memory
import sys
del sys.modules['char_dataset']
del sys.modules['cpc_model']

# Importing models
import char_dataset
import cpc_model

## 2.0. Hyperparameters

In [7]:
# ============= Hyper-parameter class for training ============== #

class Config :
    def __init__(self, seq_ctxt = True, MAXLEN = 32) : 
        self.number_of_transformer_encoders = 2
        self.number_of_attention_heads = 2
        self.hidden_size = 128
        self.dropout_prob = 0.1
        self.batch_size = 64
        self.learning_rate = 0.0003
        self.weight_decay = 0.000001
        self.no_of_epochs = 0
        self.is_causal = True # When True, the attention is causal
        self.seq_ctxt = seq_ctxt # When False, forces the context to take the beginning of the answer into account
        self.MAXLEN = MAXLEN

## 2.1. Training
Defining functions for training and testing models.

In [8]:
def train_charlm(config, device) :
    start_time = datetime.now()

    # ==================== Building datasets ================ #
    train_char_set = char_dataset.CharDataset(train_dataset, config.MAXLEN, 
                                              seq_ctxt=config.seq_ctxt)
    val_char_set = char_dataset.CharDataset(val_dataset, config.MAXLEN, 
                                            seq_ctxt=config.seq_ctxt)

    # ======================= Training ======================= #

    

    training_loader = DataLoader(train_char_set, batch_size=config.batch_size)
    validation_loader = DataLoader(val_char_set, batch_size=config.batch_size)

    charlm = cpc_model.CharLM( config, len(char_dataset.CharDataset.id_to_char), 
                              config.MAXLEN, config.is_causal).to(device)
    criterion = nn.CrossEntropyLoss()
    charlm_optimizer = optim.Adam( charlm.parameters(), lr=config.learning_rate )


    patience = 5
    best_val_loss = float('inf')

    charlm.train()
    print( datetime.now().strftime("%X"), "Training starts" )

    iteration = 0
    for epoch in tqdm(range(config.no_of_epochs)) :
        val_loss = 0.0
        for input_tensor, label in training_loader :
            input_tensor, label = input_tensor.to(device), label.to(device)
            charlm_optimizer.zero_grad()
            logits = charlm(input_tensor).to(device)
            loss = criterion(logits.squeeze(1), label)
            loss.backward()
            charlm_optimizer.step()
        iteration += 1

        print( datetime.now().strftime("%X"), "End of epoch", epoch+1, ", loss=", loss.detach().item())
        
        # Validation phase with Early Stopping
        charlm.eval()
        
        with torch.no_grad():
            for input_tensor, label in validation_loader:
                input_tensor, label = input_tensor.to(device), label.to(device)
                logits = charlm(input_tensor).to(device)
                loss = criterion(logits.squeeze(1), label)
                val_loss += loss.item()

        val_loss /= len(validation_loader)
        print(datetime.now().strftime("%X"), "Validation loss=", val_loss)

        # Check early stopping condition
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            patience = 5
        else:
            patience -= 1
            if patience == 0:
                print("Early stopping at epoch", epoch + 1)
                break

        charlm.train()
    
    end_time = datetime.now()

    return charlm, iteration + 1, end_time - start_time


In [9]:
def test_charlm(model, config, device) :

    # Computing output on the test set
    test_char_set = char_dataset.CharDataset(test_dataset, config.MAXLEN, seq_ctxt=config.seq_ctxt)
    test_loader = DataLoader(test_char_set, batch_size=config.batch_size)

    accuracies = []
    model.to(device)
    model.eval()    
    with torch.no_grad():
        for input_tensor, label in test_loader:
            input_tensor, label = input_tensor.to(device), label.to(device)
            logits = model(input_tensor).to(device)
            _, pred_label = logits.topk(1, dim=-1)
            pred_label = pred_label.squeeze(-1)  # Squeeze to remove extra dimension
            accuracy = (pred_label == label).float().mean().item()  # Compute accuracy for this batch
            accuracies.append(accuracy)  # Append batch accuracy to list)

    total_acc = np.round(np.mean(np.array(accuracies))*100, 2)
    print(f"Test accuracy : {total_acc} %")
    return total_acc

## 2.2. Grid search on parameters

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print( "Running on", device )

Running on cuda


In [11]:
output = './output/'
model_names = ['charlm_seq_32_es', 'charlm_seq_64_es', 
               'charlm_nseq_32_es', 'charlm_nseq_64_es']
configs = [Config(seq_ctxt=True, MAXLEN=32), Config(seq_ctxt=True, MAXLEN=64),
           Config(seq_ctxt=False, MAXLEN=32), Config(seq_ctxt=False, MAXLEN=64)]
test_acc = []
models = []
train_time = []
epochs = []

for config, model_name in zip(configs, model_names) :
    # Training the model
    model, epoch, delta = train_charlm(config, device)
    models.append(model)
    train_time.append(delta)
    epochs.append(epoch)


18:32:25 Training starts


  0%|          | 0/100 [00:00<?, ?it/s]

18:33:09 End of epoch 1 , loss= 1.295593023300171
18:33:13 Validation loss= 1.0855226952187045
18:33:57 End of epoch 2 , loss= 1.2957247495651245
18:34:00 Validation loss= 1.018069811917082
18:34:44 End of epoch 3 , loss= 1.20124089717865
18:34:47 Validation loss= 0.9862606394435763
18:35:33 End of epoch 4 , loss= 1.2711117267608643
18:35:36 Validation loss= 0.9662825629878661
18:36:20 End of epoch 5 , loss= 1.2029659748077393
18:36:23 Validation loss= 0.9499086381831121
18:37:08 End of epoch 6 , loss= 1.1914345026016235
18:37:11 Validation loss= 0.9377900732772664
18:37:54 End of epoch 7 , loss= 1.188745379447937
18:37:57 Validation loss= 0.93173101566421
18:38:43 End of epoch 8 , loss= 1.3086031675338745
18:38:45 Validation loss= 0.9309118220526519
18:39:30 End of epoch 9 , loss= 1.192429542541504
18:39:33 Validation loss= 0.9228368147900667
18:40:17 End of epoch 10 , loss= 1.1412465572357178
18:40:20 Validation loss= 0.9120835228628792
18:41:04 End of epoch 11 , loss= 1.074807643890

  0%|          | 0/100 [00:00<?, ?it/s]

18:45:09 End of epoch 1 , loss= 1.06986665725708
18:45:13 Validation loss= 1.1370942466988239
18:46:02 End of epoch 2 , loss= 0.8522149920463562
18:46:06 Validation loss= 1.0565689368123299
18:46:53 End of epoch 3 , loss= 0.9007502198219299
18:46:57 Validation loss= 1.0247770724980465
18:47:46 End of epoch 4 , loss= 0.8862735629081726
18:47:50 Validation loss= 1.0120429227702277
18:48:38 End of epoch 5 , loss= 0.6686652302742004
18:48:42 Validation loss= 1.004002206692668
18:49:30 End of epoch 6 , loss= 0.7730076909065247
18:49:34 Validation loss= 0.9931376493613178
18:50:23 End of epoch 7 , loss= 0.871459424495697
18:50:27 Validation loss= 0.9841223772373139
18:51:17 End of epoch 8 , loss= 0.7280705571174622
18:51:22 Validation loss= 0.9773089125776064
18:52:17 End of epoch 9 , loss= 0.7135218381881714
18:52:21 Validation loss= 0.984508383639326
18:53:14 End of epoch 10 , loss= 0.6053482294082642
18:53:18 Validation loss= 0.9861021495438479
18:54:04 End of epoch 11 , loss= 0.722262024

  0%|          | 0/100 [00:00<?, ?it/s]

18:56:36 End of epoch 1 , loss= 0.8167800307273865
18:56:39 Validation loss= 1.0656277360074808
18:57:21 End of epoch 2 , loss= 0.8002734780311584
18:57:24 Validation loss= 0.995682646153909
18:58:05 End of epoch 3 , loss= 0.7327877283096313
18:58:08 Validation loss= 0.9665966170273693
18:58:50 End of epoch 4 , loss= 0.7245216965675354
18:58:52 Validation loss= 0.9458583962709515
18:59:34 End of epoch 5 , loss= 0.7423981428146362
18:59:37 Validation loss= 0.9291363501920312
19:00:21 End of epoch 6 , loss= 0.6843223571777344
19:00:24 Validation loss= 0.9234999567663865
19:01:08 End of epoch 7 , loss= 0.6803768873214722
19:01:11 Validation loss= 0.913948885218781
19:01:56 End of epoch 8 , loss= 0.6574190258979797
19:01:59 Validation loss= 0.9085600071178815
19:02:43 End of epoch 9 , loss= 0.6175861954689026
19:02:47 Validation loss= 0.9080568722833481
19:03:31 End of epoch 10 , loss= 0.5475562214851379
19:03:34 Validation loss= 0.9013599029339248
19:04:17 End of epoch 11 , loss= 0.699652

  0%|          | 0/100 [00:00<?, ?it/s]

19:10:27 End of epoch 1 , loss= 1.2035205364227295
19:10:31 Validation loss= 1.1293165961145661
19:11:16 End of epoch 2 , loss= 0.991671085357666
19:11:21 Validation loss= 1.07066950581405
19:12:06 End of epoch 3 , loss= 1.0672489404678345
19:12:10 Validation loss= 1.044409415612319
19:12:56 End of epoch 4 , loss= 1.0152188539505005
19:13:00 Validation loss= 1.0359902235483027
19:13:45 End of epoch 5 , loss= 0.9533389210700989
19:13:49 Validation loss= 1.020416867147346
19:14:34 End of epoch 6 , loss= 0.965377151966095
19:14:38 Validation loss= 1.0163934143743247
19:15:24 End of epoch 7 , loss= 0.8445575833320618
19:15:28 Validation loss= 1.0044167009819391
19:16:13 End of epoch 8 , loss= 0.88108891248703
19:16:17 Validation loss= 1.002049796979892
19:17:03 End of epoch 9 , loss= 0.9093533754348755
19:17:07 Validation loss= 1.0015096073420162
19:17:52 End of epoch 10 , loss= 0.938504159450531
19:17:56 Validation loss= 1.005156564495265
19:18:42 End of epoch 11 , loss= 0.936672866344451

In [12]:
for model, config, model_name in zip(models, configs, model_names) :
    # Test accuracy
    accuracy = test_charlm(model, config, device)
    test_acc.append(accuracy)
    

../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [13,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [13,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [13,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [13,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [13,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [13,0,0], thread: [101,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [13,0,0],

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
for model, model_name, epoch in zip(model, model_name, epoch) :
    # Saving the model
    torch.save(model.state_dict(), f"{output}{model_name}{epoch}.pth")
    print("Model saved successfully")

Building a ``.csv`` file to store the results :

In [ ]:
import pandas as pd

data = {
    'Running time' : train_time,
    'Last epoch' : epochs,
    'Type of context' : ['Sequential' if cfg.seq_ctxt else 'Split' for cfg in configs],
    'Context size' : [cfg.MAXLEN for cfg in configs],
    'Test accuracy (%)' : test_acc
}

# Create the DataFrame with row names
df = pd.DataFrame(data, index=model_names)
df.to_csv(output + 'metrics.csv')

## 2.2. Loading a model
Now that we trained several models, we can load them.

In [ ]:
"""# Load a model
charlm = cpc_model.CharLM( config, len(char_dataset.CharDataset.id_to_char), config.MAXLEN, config.is_causal).to(device)
charlm.load_state_dict(torch.load('./output/charlm_seq_model_early_stopping.pth'))
charlm.eval()"""

# Getting the best model
best_model_idx = np.argmax(test_acc)
charlm = models[best_model_idx]
config = configs[best_model_idx]
print(f"Best model : {model_names[best_model_idx]} : \n\n", df.loc[model_names[best_model_idx]])

## 2.2. Evaluation on the best model

## 2.3. User interaction

In [ ]:
# ==================== User interaction ==================== #
while True:
    text = input("> ").strip()
    if text == "" :
        continue
    elif text == "QUIT" :
        break

    # Will be used to output question
    full_question = list(text.lower())[::-1]

    if full_question[-1] != ' ' : 
        full_question.append(' ')
    if '?' not in full_question :
        full_question = ['?'] + full_question
        
    new_character = full_question[-1]

    # Recovering the beginning of the question
    try :
        count = 0
        MAX_COUNT = 50
        while new_character != char_dataset.CharDataset.BOQ and count < MAX_COUNT :
            # Building context
            char_list = []
            if config.seq_ctxt :
                char_list = full_question[-config.MAXLEN:]
            else :
                tmp = "".join(full_question)
                words_a, words_q = tmp.split("?")[0], tmp.split("?")[1]
                words_a = list(words_a)
                len_q = len(words_q) # counting the <BOS> token
                if len_q <= config.MAXLEN//2 or len(words_a) < config.MAXLEN//8 :
                    char_list = full_question[-config.MAXLEN:]
                else :
                    char_list = words_a[config.MAXLEN//2:] + full_question[-config.MAXLEN:]
            
            ctxt = [char_dataset.CharDataset.char_to_id[c] if c in char_dataset.CharDataset.char_to_id 
                    else char_dataset.CharDataset.char_to_id[char_dataset.CharDataset.char_to_id[char_dataset.CharDataset.PADDING_SYMBOL]] 
                    for c in char_list]
            
            
            # Computing the next character
            input_tensor = torch.tensor( [0]*(config.MAXLEN-len(ctxt)) + ctxt).unsqueeze(0).to(device)
            logits = charlm(input_tensor).squeeze().to(device)
            _, new_character_tensor = logits.topk(1)
            new_character = char_dataset.CharDataset.id_to_char[new_character_tensor.detach().item()]
            
            # Uploading the final output
            full_question.append(new_character)
            count += 1

        full_question = "".join(full_question[::-1])
        print(f"Recovered question : {full_question}")
    except KeyError :
        print("ERROR")
        continue